In [53]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import cifar10
import pandas as pd


# Basics

In [10]:
tf.constant(4,  dtype = float)
tf.constant([[1,2,3,4,5,6]], shape = (1,6), dtype = float)

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=array([[1., 2., 3., 4., 5., 6.]], dtype=float32)>

In [11]:
tf.random.normal((4,3))

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.09487627, -0.71021616, -1.2129327 ],
       [-0.4790518 ,  0.95202565,  0.38115796],
       [ 0.39500883,  0.11978938,  0.49701583],
       [-0.8134479 , -0.6455141 , -0.8509691 ]], dtype=float32)>

In [13]:
x = tf.constant([1,2,3])
y = tf.constant([4,5,6])

z = tf.add(x,y)
print(z)

tf.Tensor([5 7 9], shape=(3,), dtype=int32)


In [15]:
tf.tensordot(x,y, axes=1)

<tf.Tensor: shape=(), dtype=int32, numpy=32>

In [17]:
tf.reduce_sum(x*y, axis=0)

<tf.Tensor: shape=(), dtype=int32, numpy=32>

In [18]:
x = tf.random.normal((4,3))
y = tf.random.normal((3,3))

In [19]:
tf.matmul(x,y)

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.29899675,  1.0244724 ,  0.7977205 ],
       [ 0.7262494 ,  1.9731113 ,  1.0132442 ],
       [-1.7738837 , -1.494336  ,  0.52514845],
       [-0.8479573 ,  0.00434151,  2.2959368 ]], dtype=float32)>

In [20]:
x@y

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.29899675,  1.0244724 ,  0.7977205 ],
       [ 0.7262494 ,  1.9731113 ,  1.0132442 ],
       [-1.7738837 , -1.494336  ,  0.52514845],
       [-0.8479573 ,  0.00434151,  2.2959368 ]], dtype=float32)>

In [26]:
x = tf.range(9)
x = tf.reshape(x, (3,3))
x = tf.transpose(x, perm=[1,0])
print(x)

tf.Tensor(
[[0 3 6]
 [1 4 7]
 [2 5 8]], shape=(3, 3), dtype=int32)


# Sequntial and Functional API

In [34]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [35]:
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [ ]:
model = keras.Sequential(
    [
        layers.Input(shape = (28,28,1)),
        layers.Conv2D(32, 3, activation='relu',),
        layers.MaxPool2D(pool_size=(2)),
        layers.Conv2D(64, 3, activation='relu',),
        layers.MaxPool2D(),
        layers.Conv2D(128, 3, activation='relu',),
        layers.Flatten(),
        layers.Dense(64, activation= 'relu'),
        layers.Dense(10)
    ]
)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size = 64, epochs = 10, verbose = 2)

In [39]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

79/79 - 0s - loss: 5.4916 - accuracy: 0.1010 - 164ms/epoch - 2ms/step


[5.491640090942383, 0.10100000351667404]

# Convolutional NN

In [3]:
(xtrain, ytrain), (xtest, ytest) = cifar10.load_data()
xtrain = xtrain.astype("float32") /255.0
xtest = xtest.astype("float32") /255.0

In [7]:
model = keras.Sequential(
    [
        layers.Input(shape = (32,32,3)),
        layers.Conv2D(32, (3, 3), activation='relu',),
        layers.MaxPool2D(pool_size=(2,2)),
        layers.Conv2D(64, (3, 3), activation='relu',),
        layers.MaxPool2D(),
        layers.Conv2D(128, (3, 3), activation='relu',),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ]
)

In [29]:
def my_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(32, 3, padding="same", kernel_regularizer=regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 5, padding="same", kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(128, 3,padding="same", kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs = inputs, outputs= outputs)
    return model

In [30]:
model = my_model()

In [32]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adagrad(learning_rate=3e-4),
    metrics=['accuracy']
)

model.fit(xtrain, ytrain, batch_size=64, epochs=10, verbose=2)

model.evaluate(xtest, ytest, batch_size = 64, verbose = 2)

Epoch 1/10
782/782 - 88s - loss: 1.5419 - accuracy: 0.4641 - 88s/epoch - 112ms/step
Epoch 2/10
782/782 - 87s - loss: 1.5213 - accuracy: 0.4728 - 87s/epoch - 112ms/step
Epoch 3/10
782/782 - 88s - loss: 1.5068 - accuracy: 0.4773 - 88s/epoch - 112ms/step
Epoch 4/10
782/782 - 92s - loss: 1.5063 - accuracy: 0.4766 - 92s/epoch - 117ms/step
Epoch 5/10
782/782 - 92s - loss: 1.5049 - accuracy: 0.4773 - 92s/epoch - 117ms/step
Epoch 6/10
782/782 - 88s - loss: 1.4907 - accuracy: 0.4801 - 88s/epoch - 112ms/step
Epoch 7/10
782/782 - 88s - loss: 1.4843 - accuracy: 0.4852 - 88s/epoch - 113ms/step
Epoch 8/10
782/782 - 88s - loss: 1.4903 - accuracy: 0.4809 - 88s/epoch - 113ms/step
Epoch 9/10
782/782 - 90s - loss: 1.4864 - accuracy: 0.4834 - 90s/epoch - 115ms/step
Epoch 10/10
782/782 - 90s - loss: 1.4801 - accuracy: 0.4841 - 90s/epoch - 116ms/step
157/157 - 4s - loss: 1.1956 - accuracy: 0.6666 - 4s/epoch - 22ms/step


[1.195557713508606, 0.6665999889373779]

# RNN, GRUS, LSTM and Bidirectional

In [40]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [41]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [43]:
model = keras.Sequential()
model.add(layers.Input(shape=(None, 28)))
model.add(
    layers.SimpleRNN(512, return_sequences= True, activation = 'tanh')
)
model.add(layers.SimpleRNN(512, activation='tanh'))
model.add(layers.Dense(10))

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate= 3e-4),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size= 64, epochs= 10, verbose = 2)
model.evaluate(x_test, y_test, batch_size=64, verbose = 2)

Epoch 1/10
938/938 - 85s - loss: 0.2490 - accuracy: 0.9233 - 85s/epoch - 90ms/step
Epoch 2/10
938/938 - 84s - loss: 0.1346 - accuracy: 0.9594 - 84s/epoch - 89ms/step
Epoch 3/10
938/938 - 81s - loss: 0.1079 - accuracy: 0.9671 - 81s/epoch - 86ms/step
Epoch 4/10
938/938 - 71s - loss: 0.0952 - accuracy: 0.9709 - 71s/epoch - 75ms/step
Epoch 5/10
938/938 - 80s - loss: 0.0789 - accuracy: 0.9760 - 80s/epoch - 85ms/step
Epoch 6/10
938/938 - 74s - loss: 0.0740 - accuracy: 0.9774 - 74s/epoch - 79ms/step
Epoch 7/10
938/938 - 71s - loss: 0.0723 - accuracy: 0.9776 - 71s/epoch - 75ms/step
Epoch 8/10
938/938 - 71s - loss: 0.0614 - accuracy: 0.9815 - 71s/epoch - 76ms/step
Epoch 9/10
938/938 - 71s - loss: 0.0624 - accuracy: 0.9808 - 71s/epoch - 75ms/step
Epoch 10/10
938/938 - 71s - loss: 0.0555 - accuracy: 0.9829 - 71s/epoch - 75ms/step
157/157 - 3s - loss: 0.0577 - accuracy: 0.9826 - 3s/epoch - 22ms/step


[0.05767861753702164, 0.9825999736785889]

In [47]:
model = keras.Sequential()
model.add(layers.Input(shape = (None, 28)))
model.add(
    layers.GRU(256, return_sequences = True, activation = 'tanh')
)
model.add(layers.GRU(256, activation = 'tanh'))
model.add(layers.Dense(10))

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 3e-4),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size=64, epochs =10, verbose =2)
model.evaluate(x_test, y_test, batch_size =64, verbose =2)

Epoch 1/10
938/938 - 75s - loss: 0.4426 - accuracy: 0.8512 - 75s/epoch - 80ms/step
Epoch 2/10
938/938 - 75s - loss: 0.1146 - accuracy: 0.9635 - 75s/epoch - 80ms/step
Epoch 3/10
938/938 - 71s - loss: 0.0714 - accuracy: 0.9776 - 71s/epoch - 76ms/step
Epoch 4/10
938/938 - 74s - loss: 0.0531 - accuracy: 0.9833 - 74s/epoch - 79ms/step
Epoch 5/10
938/938 - 74s - loss: 0.0422 - accuracy: 0.9865 - 74s/epoch - 79ms/step
Epoch 6/10
938/938 - 71s - loss: 0.0347 - accuracy: 0.9895 - 71s/epoch - 76ms/step
Epoch 7/10
938/938 - 63s - loss: 0.0288 - accuracy: 0.9909 - 63s/epoch - 67ms/step
Epoch 8/10
938/938 - 61s - loss: 0.0253 - accuracy: 0.9916 - 61s/epoch - 65ms/step
Epoch 9/10
938/938 - 65s - loss: 0.0205 - accuracy: 0.9934 - 65s/epoch - 69ms/step
Epoch 10/10
938/938 - 63s - loss: 0.0173 - accuracy: 0.9944 - 63s/epoch - 67ms/step
157/157 - 4s - loss: 0.0413 - accuracy: 0.9883 - 4s/epoch - 23ms/step


[0.04133369401097298, 0.9883000254631042]

In [48]:
model = keras.Sequential()
model.add(layers.Input(shape = (None, 28)))
model.add(
    layers.LSTM(256, return_sequences = True, activation = 'tanh')
)
model.add(layers.LSTM(256, activation = 'tanh'))
model.add(layers.Dense(64))

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits =True),
    optimizer= keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size = 64, epochs = 10, verbose = 2)
model.evaluate(x_test, y_test, batch_size =64, verbose= 2)

Epoch 1/10
938/938 - 86s - loss: 0.5005 - accuracy: 0.8340 - 86s/epoch - 91ms/step
Epoch 2/10
938/938 - 83s - loss: 0.1118 - accuracy: 0.9662 - 83s/epoch - 88ms/step
Epoch 3/10
938/938 - 83s - loss: 0.0743 - accuracy: 0.9775 - 83s/epoch - 89ms/step
Epoch 4/10
938/938 - 78s - loss: 0.0551 - accuracy: 0.9823 - 78s/epoch - 84ms/step
Epoch 5/10
938/938 - 78s - loss: 0.0445 - accuracy: 0.9865 - 78s/epoch - 83ms/step
Epoch 6/10
938/938 - 80s - loss: 0.0385 - accuracy: 0.9885 - 80s/epoch - 86ms/step
Epoch 7/10
938/938 - 76s - loss: 0.0308 - accuracy: 0.9903 - 76s/epoch - 81ms/step
Epoch 8/10
938/938 - 76s - loss: 0.0305 - accuracy: 0.9902 - 76s/epoch - 82ms/step
Epoch 9/10
938/938 - 73s - loss: 0.0249 - accuracy: 0.9920 - 73s/epoch - 78ms/step
Epoch 10/10
938/938 - 73s - loss: 0.0216 - accuracy: 0.9932 - 73s/epoch - 78ms/step
157/157 - 4s - loss: 0.0469 - accuracy: 0.9864 - 4s/epoch - 28ms/step


[0.04690316691994667, 0.9864000082015991]

In [50]:
model = keras.Sequential()
model.add(layers.Input(shape = (None, 28)))
model.add(
    layers.Bidirectional(
        layers.LSTM(256, return_sequences = True, activation = 'tanh')
    )
)
model.add(
    layers.Bidirectional(
        layers.LSTM(256, activation = 'tanh')
    )
)
model.add(layers.Dense(10))

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)
model.fit(x_train, y_train, batch_size = 64, epochs = 10, verbose = 2)
model.evaluate(x_test, y_test, batch_size =64, verbose =2)

Epoch 1/10
938/938 - 244s - loss: 0.2737 - accuracy: 0.9102 - 244s/epoch - 260ms/step
Epoch 2/10
938/938 - 236s - loss: 0.0775 - accuracy: 0.9765 - 236s/epoch - 251ms/step
Epoch 3/10
938/938 - 235s - loss: 0.0544 - accuracy: 0.9829 - 235s/epoch - 251ms/step
Epoch 4/10
938/938 - 269s - loss: 0.0433 - accuracy: 0.9868 - 269s/epoch - 287ms/step
Epoch 5/10
938/938 - 273s - loss: 0.0360 - accuracy: 0.9890 - 273s/epoch - 291ms/step
Epoch 6/10
938/938 - 253s - loss: 0.0288 - accuracy: 0.9910 - 253s/epoch - 270ms/step
Epoch 7/10
938/938 - 243s - loss: 0.0248 - accuracy: 0.9924 - 243s/epoch - 259ms/step
Epoch 8/10
938/938 - 243s - loss: 0.0215 - accuracy: 0.9932 - 243s/epoch - 259ms/step
Epoch 9/10
938/938 - 243s - loss: 0.0188 - accuracy: 0.9941 - 243s/epoch - 259ms/step
Epoch 10/10
938/938 - 244s - loss: 0.0149 - accuracy: 0.9950 - 244s/epoch - 260ms/step
157/157 - 13s - loss: 0.0442 - accuracy: 0.9880 - 13s/epoch - 82ms/step


[0.04417228326201439, 0.9879999756813049]

# Functional API for Multilabel multiclass classification

In [62]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [76]:
train_df = pd.read_csv("mnist_multi_digit_/train.csv")
test_df = pd.read_csv("mnist_multi_digit_/test.csv")

train_images = os.getcwd() + "/mnist_multi_digit_/" + "train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/mnist_multi_digit_/" + "test_images/" + test_df.iloc[:, 0].values

In [77]:
train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [78]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    labels = {"first_number": label[0], "second_number": label[1]}
    return image, labels

In [79]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.shuffle(buffer_size=len(test_labels))
    .map(read_image)
    .batch(batch_size = BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [80]:
inputs = keras.Input(shape = (64,64,1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding='same',
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(
    64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(
    64, 
    3,
    activation = 'relu',
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
output1 = layers.Dense(10, activation='softmax', name='first_number')(x)
output2 = layers.Dense(10, activation='softmax', name='second_number')(x)

model = keras.Model(inputs=inputs, outputs=[output1, output2])
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=[
        keras.losses.SparseCategoricalCrossentropy(),
        keras.losses.SparseCategoricalCrossentropy(),
    ],
    metrics=['accuracy'],
)

model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)




Epoch 1/5
1000/1000 - 728s - loss: 1.7751 - first_number_loss: 0.8600 - second_number_loss: 0.8300 - first_number_accuracy: 0.6999 - second_number_accuracy: 0.7075 - 728s/epoch - 728ms/step
Epoch 2/5
1000/1000 - 719s - loss: 0.5556 - first_number_loss: 0.2483 - second_number_loss: 0.2406 - first_number_accuracy: 0.9195 - second_number_accuracy: 0.9222 - 719s/epoch - 719ms/step
Epoch 3/5
1000/1000 - 731s - loss: 0.3561 - first_number_loss: 0.1514 - second_number_loss: 0.1462 - first_number_accuracy: 0.9520 - second_number_accuracy: 0.9538 - 731s/epoch - 731ms/step
Epoch 4/5
1000/1000 - 777s - loss: 0.2792 - first_number_loss: 0.1146 - second_number_loss: 0.1111 - first_number_accuracy: 0.9636 - second_number_accuracy: 0.9640 - 777s/epoch - 777ms/step
Epoch 5/5
